<a href="https://colab.research.google.com/github/pedrobcavalcante/Backtest-trading-strategies/blob/main/Gerador_de_Par%C3%A2metros_para_estrat%C3%A9gias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [65]:
import backtesting 
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import os.path
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
backtesting.set_bokeh_output(notebook=False)


In [66]:
from backtesting.test import SMA


In [67]:
def fixData(data):
    columns = ['open_time', 'Open', 'High', 'Low', 'Close', 'Volume',
               'close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore']
    df = pd.DataFrame(data, columns=columns)
    df.index = [dt.datetime.fromtimestamp(x/1000.0).date() for x in df.close_time]
    df.sort_values(["open_time"], ascending=False)
    df = df.astype('double')
    df['Volume'] = df['Volume'].astype('int')
    df = df.drop(['qav', 'num_trades', 'taker_base_vol',
                 'taker_quote_vol', 'ignore'], axis=1)
    return df


In [68]:
def get_bars(symbol, interval='1h'):
    root_url = 'https://api.binance.com/api/v1/klines'
    filename = "data/"+symbol+"_"+interval+".csv"
    url = root_url + '?symbol=' + symbol + '&interval=' + interval+"&limit=1001"
    print(url)
    data = json.loads(requests.get(url).text)
    df = fixData(data)
#    df.to_csv(filename, sep=';')
    return df


In [69]:
class SmaCross(Strategy):
    n1 = 0
    n2 = 0

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.High, self.n1)
        self.sma2 = self.I(SMA, self.data.Low, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.data.Close, self.sma1):
            # self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.data.Close):
            self.position.close()
            # self.sell()

    def setInitialParams(self, valor1, valor2):
        self.n1 = valor1
        self.n2 = valor2


# Nova seção


In [70]:

data = get_bars("BTCUSDT", "1d")
SmaCross.setInitialParams(SmaCross,5,5)
bt = Backtest(data, SmaCross,cash=1000000,exclusive_orders=False)
output = bt.run()
print(output)
bt.plot()


https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1d&limit=1001


/home/pedro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  This is separate from the ipykernel package so we can avoid doing imports until


Start                              2019-01-30
End                                2021-10-25
Duration                    999 days 00:00:00
Exposure Time [%]                          66
Equity Final [$]                  8.10698e+06
Equity Peak [$]                   8.81364e+06
Return [%]                            710.698
Buy & Hold Return [%]                 1726.16
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                                0
Max. Drawdown [%]                    -42.1184
Avg. Drawdown [%]                     -8.1151
Max. Drawdown Duration      489 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                  113
Win Rate [%]                          53.9823
Best Trade [%]                        37.3781
Worst Trade [%]                      -21.1858
Avg. Trade [%]                    

Row(id='6491', ...)